## load modules

Remember to set model name. OpenAI updates model name and the old ones may become invalid

e.g. gpt-4o-mini-2024-07-18

In [ ]:
import sys
!{sys.executable} -m pip install ollama pandas tqdm python-dotenv

import os
import pandas as pd
from tqdm import tqdm
from pathlib import Path
import json
import ollama


[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python3.11 -m pip install --upgrade pip


In [ ]:
import pandas as pd
from tqdm import tqdm
from pathlib import Path
import os
import json


# Point to the folder path
input_folder = Path("Phase2_Folders/030_processed_html")

output_folder= "USHMM-PDF-cleansing/Gender_family_topics_identification/topic_labeling"
# Ensure output folder exists
os.makedirs(output_folder, exist_ok=True)

class File:
    def __init__(self, file_path):
        self.file_path = file_path
        self.topics = {}


# Dictionary to store results
files = {}


def safe_json_loads(s):
    """Attempts to fix and parse a possibly incomplete JSON string."""
    s = s.strip()
    
    # Basic brace balancing
    open_braces = s.count('{')
    close_braces = s.count('}')
    
    if close_braces < open_braces:
        s += '}' * (open_braces - close_braces)
    
    try:
        return json.loads(s)
    except json.JSONDecodeError as e:
        print(f"JSON parsing failed: {e}")
        return None

def single_csv_classification(file_obj, output_path):
    """Process a single CSV file and classify topics for each row."""
    df = pd.read_csv(file_obj.file_path)
    df["topics"] = ""

    for i, row in tqdm(df.iterrows(), total=len(df)):
        try:
            question = row["Question"]
            answer = row["Answer"]
            #print(question+"\n"+answer)
            payload = generate_payload(question, answer)
            #print(payload)
            topics = content_request(payload)
            #topics=[1,2,3]
            df.at[i, "topics"] = topics
            
            print(topics)
            
            temp = safe_json_loads(topics)
            if temp:
                # Extract only the topic strings (e.g., temp["topic1"]["topic"])
                for key in temp:
                    if isinstance(temp[key], dict) and "topic" in temp[key]:
                        topic_str = temp[key]["topic"]
                        file_obj.topics[topic_str] = file_obj.topics.get(topic_str, 0) + 1

            

            # Count topic frequencies
            for key, value in temp.items():
                print(f"{key}: {value}")
                #file_obj.topics[key] = file_obj.topics.get(topic, 0) + 1

        except Exception as e:
            print(f"Error at row {i}: {e}")
            df.at[i, "topics"] = []

    # Save to file
    filename = Path(file_obj.file_path).stem
    df.to_csv(f"{output_path}/{filename}_labeled.csv", index=False)



n=1
# Loop through all CSV files in the folder
for file_path in input_folder.glob("*.csv"):
    if(n==1):
        break 
    n=n+1

    file_obj = File(file_path)
    single_csv_classification(file_obj, output_folder)
    files[file_path.name] = file_obj  # Save the object to access topic counts later


file_obj = File("USHMM-PDF-cleansing/Phase2_Folders/030_processed_html/RG-50.030.0038_trs_en_cleaned.csv")
single_csv_classification(file_obj, output_folder)
files[file_path.name] = file_obj  # Save the object to access topic counts later


# Example: Print topic counts from all files
for fname, file_obj in list(files.items())[:1]:
    print(f"\n {fname}")
    #print(file_obj.topics)
    for topic, count in file_obj.topics.items():
        print(f"  {topic}: {count}")

  1%|▏         | 1/75 [00:43<54:10, 43.93s/it]

<think>
Okay, so the user has provided a JSON structure with three topic slots. They want me to fill these based on an interview response about someone's name and birthplace. The topics available are a bunch of historical ones related to the Holocaust.

First, looking at the response: "My name is Leo Bretholz. Do I have to spell that? I usually do have to. And I was born in Vienna, Austria." This seems like personal information about someone's name and place of birth. 

I need to figure out which topics this fits into. The first topic should be the most relevant. Since the user asked for a name and birthplace, it directly relates to "Biography or life story" but that's not in the provided list. Wait, let me check the available topics again.

Looking through the list: 'Jews and Jewish identity...' could be related if Leo Bretholz was Jewish, which I'm assuming given the context of Holocaust-related conversations. But without knowing for sure, it might be a stretch. Alternatively, "Where

  3%|▎         | 2/75 [00:53<28:58, 23.81s/it]

<think>
Okay, let me try to figure this out. So the user has provided some data about Holocaust education and a JSON response based on an interview question asking for the year when something happened. Specifically, they mentioned that a witness named Anna survived Auschwitz in 1943.

Now, I need to create another JSON response following the same structure but with different topics. The task is to generate a response where topic1 is different from the previous one and then include two more relevant topics.

First, looking at the previous example: topic1 was about Auschwitz, which made sense because it's directly related to the event. So for this new question about concentration camps in general, topic1 should be something like "Concentration camps (general)" or another broad category that includes all types of camps used by Nazi Germany.

Next, topic2 and topic3 need to cover other relevant areas. Since the user has a long list of topics, I can pick another area, maybe "Numbers" becaus

  4%|▍         | 3/75 [01:07<22:50, 19.04s/it]

...

<think>
Alright, let's tackle this query step by step. The user has provided a JSON structure that includes three topics related to the Holocaust and has asked for a response tagging their conversation based on these topics.

First, I need to understand the context. The interview question is about where someone went, specifically mentioning Trier in Germany. This falls under 'Germany, Germans' as it's directly referring to a location associated with the Holocaust.

Next, I should consider why this location might be significant. Trier was part of the German territories that were occupied and subjected to forced labor during the Nazi era. This makes it highly relevant to the topic of 'Germany, Germans'.

The third topic needs to be something else but still related. Perhaps a nearby place like Koblenz could make sense here since it's also part of the occupied region and was used in concentration camps.

I should ensure that each topic is distinct from the others, so I'll assign different 

 13%|█▎        | 10/75 [02:11<09:46,  9.02s/it]

<think>
Okay, so I need to create a JSON response based on this user's query. Let me read through the entire text first to understand what it's about.

The user provided a detailed narrative starting with a request for a JSON response in the form of a story about a young man escaping Nazi Germany and making his way to Israel. It mentions various steps like moving from Luxembourg to Belgium, crossing into France, and ending up in Baltimore. There are also references to events like Kristallnacht.

Hmm, the user's query seems to be asking for a JSON response based on this narrative but it doesn't explicitly state what exactly they want in the JSON format. They just wrote "Create a JSON response" after providing the story. I need to figure out what information should be included in the JSON.

Looking at the structure of typical JSON responses, it's usually an array or an object containing key-value pairs. Since this is a narrative with multiple entities like names, places, events, etc., pe

 13%|█▎        | 10/75 [02:11<14:17, 13.19s/it]


KeyboardInterrupt: 

In [ ]:
import pandas as pd
from tqdm import tqdm
from pathlib import Path
import os
import json


# Point to the folder path
input_folder = Path("Phase2_Folders/030_processed_html")

output_folder= "USHMM-PDF-cleansing/Gender_family_topics_identification/topic_labeling"
# Ensure output folder exists
os.makedirs(output_folder, exist_ok=True)

class File:
    def __init__(self, file_path):
        self.file_path = file_path
        self.topics = {}


# Dictionary to store results
files = {}


def safe_json_loads(s):
    """Attempts to fix and parse a possibly incomplete JSON string."""
    s = s.strip()
    
    # Basic brace balancing
    open_braces = s.count('{')
    close_braces = s.count('}')
    
    if close_braces < open_braces:
        s += '}' * (open_braces - close_braces)
    
    try:
        return json.loads(s)
    except json.JSONDecodeError as e:
        print(f"JSON parsing failed: {e}")
        return None

def single_csv_classification(file_obj, output_path):
    """Process a single CSV file and classify topics for each row."""
    df = pd.read_csv(file_obj.file_path)
    df["topics"] = ""

    for i, row in tqdm(df.iterrows(), total=len(df)):
        try:
            question = row["Question"]
            answer = row["Answer"]
            #print(question+"\n"+answer)
            payload = generate_payload(question, answer)
            #print(payload)
            topics = content_request(payload)
            #topics=[1,2,3]
            df.at[i, "topics"] = topics
            
            print(topics)
            
            temp = safe_json_loads(topics)
            if temp:
                # Extract only the topic strings (e.g., temp["topic1"]["topic"])
                for key in temp:
                    if isinstance(temp[key], dict) and "topic" in temp[key]:
                        topic_str = temp[key]["topic"]
                        file_obj.topics[topic_str] = file_obj.topics.get(topic_str, 0) + 1

            

            # Count topic frequencies
            for key, value in temp.items():
                print(f"{key}: {value}")
                #file_obj.topics[key] = file_obj.topics.get(topic, 0) + 1

        except Exception as e:
            print(f"Error at row {i}: {e}")
            df.at[i, "topics"] = []

    # Save to file
    filename = Path(file_obj.file_path).stem
    df.to_csv(f"{output_path}/{filename}_labeled.csv", index=False)



n=1
# Loop through all CSV files in the folder
for file_path in input_folder.glob("*.csv"):
    if(n==1):
        break 
    n=n+1

    file_obj = File(file_path)
    single_csv_classification(file_obj, output_folder)
    files[file_path.name] = file_obj  # Save the object to access topic counts later


file_obj = File("USHMM-PDF-cleansing/Phase2_Folders/030_processed_html/RG-50.030.0038_trs_en_cleaned.csv")
single_csv_classification(file_obj, output_folder)
files[file_path.name] = file_obj  # Save the object to access topic counts later


# Example: Print topic counts from all files
for fname, file_obj in list(files.items())[:1]:
    print(f"\n {fname}")
    #print(file_obj.topics)
    for topic, count in file_obj.topics.items():
        print(f"  {topic}: {count}")

  1%|▏         | 1/75 [00:43<54:10, 43.93s/it]

<think>
Okay, so the user has provided a JSON structure with three topic slots. They want me to fill these based on an interview response about someone's name and birthplace. The topics available are a bunch of historical ones related to the Holocaust.

First, looking at the response: "My name is Leo Bretholz. Do I have to spell that? I usually do have to. And I was born in Vienna, Austria." This seems like personal information about someone's name and place of birth. 

I need to figure out which topics this fits into. The first topic should be the most relevant. Since the user asked for a name and birthplace, it directly relates to "Biography or life story" but that's not in the provided list. Wait, let me check the available topics again.

Looking through the list: 'Jews and Jewish identity...' could be related if Leo Bretholz was Jewish, which I'm assuming given the context of Holocaust-related conversations. But without knowing for sure, it might be a stretch. Alternatively, "Where

  3%|▎         | 2/75 [00:53<28:58, 23.81s/it]

<think>
Okay, let me try to figure this out. So the user has provided some data about Holocaust education and a JSON response based on an interview question asking for the year when something happened. Specifically, they mentioned that a witness named Anna survived Auschwitz in 1943.

Now, I need to create another JSON response following the same structure but with different topics. The task is to generate a response where topic1 is different from the previous one and then include two more relevant topics.

First, looking at the previous example: topic1 was about Auschwitz, which made sense because it's directly related to the event. So for this new question about concentration camps in general, topic1 should be something like "Concentration camps (general)" or another broad category that includes all types of camps used by Nazi Germany.

Next, topic2 and topic3 need to cover other relevant areas. Since the user has a long list of topics, I can pick another area, maybe "Numbers" becaus

  4%|▍         | 3/75 [01:07<22:50, 19.04s/it]

...

<think>
Alright, let's tackle this query step by step. The user has provided a JSON structure that includes three topics related to the Holocaust and has asked for a response tagging their conversation based on these topics.

First, I need to understand the context. The interview question is about where someone went, specifically mentioning Trier in Germany. This falls under 'Germany, Germans' as it's directly referring to a location associated with the Holocaust.

Next, I should consider why this location might be significant. Trier was part of the German territories that were occupied and subjected to forced labor during the Nazi era. This makes it highly relevant to the topic of 'Germany, Germans'.

The third topic needs to be something else but still related. Perhaps a nearby place like Koblenz could make sense here since it's also part of the occupied region and was used in concentration camps.

I should ensure that each topic is distinct from the others, so I'll assign different 

 13%|█▎        | 10/75 [02:11<09:46,  9.02s/it]

<think>
Okay, so I need to create a JSON response based on this user's query. Let me read through the entire text first to understand what it's about.

The user provided a detailed narrative starting with a request for a JSON response in the form of a story about a young man escaping Nazi Germany and making his way to Israel. It mentions various steps like moving from Luxembourg to Belgium, crossing into France, and ending up in Baltimore. There are also references to events like Kristallnacht.

Hmm, the user's query seems to be asking for a JSON response based on this narrative but it doesn't explicitly state what exactly they want in the JSON format. They just wrote "Create a JSON response" after providing the story. I need to figure out what information should be included in the JSON.

Looking at the structure of typical JSON responses, it's usually an array or an object containing key-value pairs. Since this is a narrative with multiple entities like names, places, events, etc., pe

 13%|█▎        | 10/75 [02:11<14:17, 13.19s/it]


KeyboardInterrupt: 

## Classification

In [ ]:
# Corrected candidate topics list 
candidate_topics = [
    "Auschwitz",
    "Baltic states and Sweden",
    "Boder trying to understand reasons, skepticism",
    "Christians, church, religion",
    "France, French",
    "Germany, Germans",
    "Interactions with SS and SS actions",
    "Interactions with and feelings about Germans, Aryan identities",
    "Italy",
    "Jews and Jewish identity, false papers",
    "Jews and fate of Jewish community",
    "Palestine, Judaism, kibbutzim",
    "Paris, France",
    "Poland, Polish",
    "Russia, Russians",
    "Switzerland",
    "To and from cities and countries",
    "Actions, choices, and intentions",
    "Arrest, beating",
    "Behavior, treatment, and aftermath",
    "Birth, living situation, camp life",
    "Children",
    "Clarifications, mostly geographic and people",
    "Closed-ended, factual questions, mostly yes-no questions",
    "Clothing and material objects",
    "Collective actions vs. individual agency",
    "Countryside (forest, fields, and farms)",
    "Days, daily life, daily routines",
    "Encounters with Germans",
    "Escaping, leaving, being evacuated, fleeing",
    "Factory work, forced labor, business",
    "Factual clarification (name, address, number, meaning)",
    "Family separation",
    "Family (background, what happened to)",
    "Food, eating, and drinking",
    "Fuel",
    "Geographic locations, camp names/locations",
    "Ghetto experiences",
    "Hospital, doctor",
    "How long (lengths of time)",
    "How many (people)",
    "How many people in family",
    "How old (age)",
    "Hygiene, toilet facilities, body violations",
    "Introductions, storytelling",
    "Killing, shooting, and death",
    "Lager (names, conditions, factual details)",
    "Liberation, end of war",
    "Living quarters (homes, barracks)",
    "Locations and activities in streets, cities",
    "Longer questions about unique Holocaust experiences",
    "Marriage, marital relations, family",
    "Men (fathers, sons, brothers, husbands, uncles)",
    "Men and women",
    "Military, soldiers, and construction",
    "Mistreatment and death, corpses, gassings, crematoria",
    "Mistreatment and reactions",
    "Music, song",
    "Night",
    "Numerical quantities (mostly people)",
    "Numerical quantities and dates",
    "Outbreak of the war, events of war",
    "Payments and wages",
    "Prisoners",
    "Professional work and industry",
    "Quantities (size, weight, amounts, distance)",
    "Questions about men",
    "Questions of meaning",
    "References to women",
    "Schooling",
    "Sexual violence and assult",
    "Short negation questions (factual)",
    "Short questions about processes",
    "Shorter durations of time (hours, days, weeks, months)",
    "Sickness, disease, illness",
    "Sleeping arrangements, beds",
    "Smoking, cigarettes, bartering",
    "Studying, learning",
    "Tattoos",
    "Tell me/describe",
    "Time (month, year), when",
    "Transportation and deportation",
    "War destruction",
    "What",
    "What happened (next)",
    "What, how",
    "Where",
    "Who",
    "Why, how",
    "Witnessing events",
    "Women (mothers, daughters, sisters, wives, aunts)",
    "Writing/language/inscriptions"
]

class File:
    def __init__(self, file_path):
        self.file_path = file_path
        self.topics = {}


def build_structured_response(question, answer):
    """Classify topics using Ollama's DeepSeek model with an improved prompt."""
    system_message = "You are a helpful assistant tasked with tagging conversations based on their content."
    user_message = f"""
    Analyze the following question and answer, then identify 1 to 3 relevant topics from the provided list.

    Question: {question}
    Answer: {answer}

    Candidate topics:
    {', '.join(candidate_topics)}

    Instructions:
    - Select 1 to 3 topics that best match the content.
    - Respond with a JSON object containing a single key 'relevant_topics' and an array of the selected topics.
    - Ensure the response is a valid JSON object with no additional text or explanations.

    Example response:
    {{"relevant_topics": ["Education", "Childhood"]}}
    """

    try:
        # Call Ollama API with DeepSeek model
        response = ollama.chat(
            model="deepseek-coder",
            messages=[
                {"role": "system", "content": system_message},
                {"role": "user", "content": user_message}
            ],
            options={"format": "json", "temperature": 0.2}  # Lower temperature for consistency
        )
        
        # Extract the response content and debug
        json_response = response["message"]["content"]
        print(f"Raw response: {json_response}")  # Print raw output for inspection
        
        # Parse the response as JSON
        data = json.loads(json_response)
        
        # Validate and return relevant topics
        if "relevant_topics" in data and isinstance(data["relevant_topics"], list):
            return data["relevant_topics"]
        else:
            print("Warning: 'relevant_topics' not found or invalid in response.")
            return []
            
    except json.JSONDecodeError:
        print("Error: Model response is not valid JSON.")
        return []
    except Exception as e:
        print(f"Error processing request: {e}")
        return []

def single_csv_classification(file_obj, output_folder):
    """Process a single CSV file and classify topics for each row."""
    df = pd.read_csv(file_obj.file_path)
    df["topics"] = ""

    for i, row in tqdm(df.iterrows(), total=len(df)):
        try:
            question = row["Question"]
            answer = row["Answer"]
            topics = build_structured_response(question, answer)
            df.at[i, "topics"] = topics

            # Count topic frequencies
            for topic in topics:
                file_obj.topics[topic] = file_obj.topics.get(topic, 0) + 1

        except Exception as e:
            print(f"Error at row {i}: {e}")
            df.at[i, "topics"] = []

    # Save to file
    filename = Path(file_obj.file_path).stem
    df.to_csv(f"{output_folder}/{filename}_labeled.csv", index=False)

In [5]:
# Test with a single file
folder_path = Path("Phase2_Folders/030_processed_html")
output_folder = "/Users/xiyuancao/Desktop/USHMM-PDF-cleansing/Gender_family_topics_identification/topic_labeling"
os.makedirs(output_folder, exist_ok=True)

files = {}
csv_files = list(folder_path.glob("*.csv"))

if csv_files:
    file_path = csv_files[0]  # Process only the first CSV file
    file_obj = File(file_path)
    single_csv_classification(file_obj, output_folder)
    files[file_path.name] = file_obj

    # Print topic counts
    for fname, file_obj in files.items():
        print(f"\n📄 {fname}")
        for topic, count in file_obj.topics.items():
            print(f"  {topic}: {count}")

  1%|          | 1/131 [00:02<06:16,  2.90s/it]

Raw response: Based on your question, it seems like you've provided an answer and asked for topics related to that content which is not a clear cut scenario as there are no given candidates or questions. However if we assume the context of these words in relation with each other (like Doris being born), then "Doris" could be considered more relevant than its gender role, family structure etc., based on common knowledge and usage patterns among people familiar to us from a social science perspective.

In this case: 1) Education is related mostly because of the educational background or training given by Doris; while childhood might relate less as it's not directly connected with education but rather her childhood experiences, which could be seen in family relationships and personal development activities that she underwent during these periods. Therefore we can say "Education" would likely appear first among our list of relevant topics based on common knowledge patterns from a social sc

  2%|▏         | 2/131 [00:03<03:33,  1.65s/it]

Raw response: Based on your question, it seems like you've provided an answer and asked for topics related to that content which is a bit ambiguous as there are multiple interpretations of the same statement. However if we assume Dora Costinski was talking about her parents then relevant topic would be "Family structure", because she mentioned family separation (which could also refer back to childhood), but not specifically mentioning any other topics related with parenting or marriage and divorce which is a bit outdated in the context of this question.

Error: Model response is not valid JSON.


  2%|▏         | 3/131 [00:05<04:02,  1.89s/it]

...

Raw response: Based on your question, it seems like you've provided an answer and asked for topics that match this content from the list of potential candidate topics given in a JSON object. Here is how I would respond to such query using Python programming language with its json module which provides methods to work easily with JSON data structures as dictionaries or lists:
```python
import json  # Importing python's built-in 'json' library for handling the task of converting between different types in a simple and efficient way.
# Defining your list/candidate topics, you can add more if needed by following similar pattern below line "Candidates_topics" is defined as:  Gender roles...etc are added here too but they're not relevant to the given question content so I left them out for brevity.
# Candidates_Topics = ["Gender Roles", ...] # Add more if needed, you can add all topics from your list in this line of code 
Candidate_topics=["Family Structure","Parenting"]  	# Define the candi

 40%|███▉      | 52/131 [01:40<02:47,  2.12s/it]

Raw response: Based on your question, it seems like you've provided an answer and asked for topics related to that content (i.e., "Gender roles"). Here is a possible JSON object response based off the given information:
```json
{
    "relevant_topics": ["Education", "Childhood"] 
}
```
This suggests two relevant areas of study in this context - Education and Childhood, which are topics related to gender roles. However, please note that as an AI model developed for answering questions based on a given set of data (in your case the provided answer), my responses might not be accurate or comprehensive if they rely heavily upon external information such as specific contexts from other sources you didn't ask about in this question.

Error: Model response is not valid JSON.


 40%|███▉      | 52/131 [03:45<05:41,  4.33s/it]


KeyboardInterrupt: 

In [ ]:
# Point to the folder path
folder_path = Path("Phase2_Folders/030_processed_html")

output_folder= "/Users/xiyuancao/Desktop/USHMM-PDF-cleansing/Gender_family_topics_identification/topic_labeling"
# Ensure output folder exists
os.makedirs(output_folder, exist_ok=True)

# Dictionary to store results
files = {}

# Loop through all CSV files in the folder
for file_path in folder_path.glob("*.csv"):
    file_obj = File(file_path)
    single_csv_classification(file_obj,output_folder)
    files[file_path.name] = file_obj  # Save the object to access topic counts later

# Example: Print topic counts from all files
for fname, file_obj in files.items():
    print(f"\n {fname}")
    for topic, count in file_obj.topics.items():
        print(f"  {topic}: {count}")

 14%|█▎        | 18/131 [00:21<02:18,  1.22s/it]


KeyboardInterrupt: 